# Tow-Based Discontinuous Composite (TBDC) property predictor
This code is written for an MSc research project at Imperial College London

## Mount drive

In [1]:
# Data to be imported from google drive
from google.colab import drive
drive.mount('/content/drive')

# Set path for drive with datasets
drive_path ="/content/drive/MyDrive/Individual_project_ICL"

Mounted at /content/drive


## Imports and installs

In [2]:
import sys
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas

In [3]:
import os
import random
import time
import math
import datetime
import shutil
import json

import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Import data


In [5]:
if 'data' not in os.listdir():
  !cp {drive_path+"/Data/Unnotched_TBDC_2022.zip"} .
  !unzip -q 'Unnotched_TBDC_2022.zip' -d data
  !rm 'Unnotched_TBDC_2022.zip'

Verify that data is imported correctly

In [361]:
def loadSample(path = str):
  '''
  Imports data in csv and formats into a tensor
  '''
  # Read sample csv data
  sample = pd.read_csv(path)
  headers = sample.columns.values.tolist()
  values = np.array(sample)

  coords = [[x for x in values[:,1][y].split(' ') if x] for y in range(len(values[:,1]))] # Split coordinates by delimiter (space)
  coords = [np.char.strip(x, '[') for x in coords] # Coordinate output from abaqus has leading "["
  coords = [[x for x in coords[y] if x] for y in range(len(values[:,1]))] # remove empty array elements
  coords = np.array([[float(x) for x in coords[y][0:2]] for y in range(len(values[:,1]))]) # Take 2d coordinates and convert to float

  values = np.column_stack((values[:,0],coords,values[:,2:])) # Create a new values vector which contains the coordinates

  headers = np.concatenate(([[headers[0],'x_coord','y_coord'],headers[2:]])) # rectify the headers to include x and y coordinates separately

  return headers, values




In [363]:
testHeaders, testSample = loadSample('data/Unnotched_TBDC_2022_0.csv')
print(testHeaders)
print(testSample)

['label' 'x_coord' 'y_coord' 'e11' 'e22' 'e12' 'S11' 'S22' 'S12' 'SMises'
 'FI' 'E11' 'E22' 'E12']
[[1 23.75 67.74545 ... 42386.6914062 43293.0625 20660.1074219]
 [2 21.25 67.74545 ... 41920.3984375 41622.0039062 21304.8339844]
 [3 18.75 67.74545 ... 41454.1054688 39950.9414062 21949.5605469]
 ...
 [1098 -18.75 -67.74545 ... 55898.6875 44675.3554688 17008.9414062]
 [1099 -21.25 -67.74545 ... 56577.2109375 42843.3398438 17448.8691406]
 [1100 -23.75 -67.74545 ... 57255.7304688 41011.3242188 17888.7988281]]
